In [4]:
import torch

#### Convolution layers

##### unfold

https://stackoverflow.com/questions/53972159/how-does-pytorchs-fold-and-unfold-work

torch.nn.Unfold(  
    kernel_size,  
    dilation=1, padding=0, stride=1)  
  
* kernel_size (int or tuple) – the size of the sliding blocks  
  
  
* stride (int or tuple, optional) – the stride of the sliding blocks in the input spatial dimensions. Default: 1
* padding (int or tuple, optional) – implicit zero padding to be added on both sides of input. Default: 0
* dilation (int or tuple, optional) – a parameter that controls the stride of elements within the neighborhood. Default: 1  


In [5]:
import torch.nn as nn

1. unfold is a making window (kernel)

In [10]:
unfold = nn.Unfold(kernel_size=(2, 3)) # 사이즈 : 2x3, 6개의 요소

In [7]:
x = torch.randn(2, 5, 3, 4)
x

tensor([[[[-1.5223, -0.2952,  0.3551,  0.2865],
          [ 0.0066, -0.6348, -1.2648, -0.3411],
          [ 0.1455,  0.6420, -1.2262,  0.6646]],

         [[ 0.5227,  0.0675,  2.2939, -0.9430],
          [ 1.7125,  0.7122, -0.6256,  1.7609],
          [ 0.6527,  2.1585, -1.9699, -1.4661]],

         [[-0.4349,  0.4933, -1.7456,  0.0696],
          [-0.6547,  0.1629, -0.0294,  1.2622],
          [-0.2090, -0.0254, -0.5135, -1.0149]],

         [[-1.9260,  1.1769, -1.1372, -0.2015],
          [ 0.6423, -0.6941, -1.6763,  0.9137],
          [ 1.8009,  0.7254, -0.0135,  0.1131]],

         [[-1.1822, -1.4841,  0.5706, -0.2356],
          [ 0.3818,  0.6736,  0.5526,  0.1307],
          [-1.5265,  0.1199, -1.3868,  1.5758]]],


        [[[-1.0304, -0.1849,  0.5200,  0.8863],
          [ 0.4583,  0.7302, -0.2049, -0.5554],
          [-1.2646,  0.1102, -0.3027,  0.1845]],

         [[ 0.4840,  0.8333, -0.6412,  0.1023],
          [ 1.3409,  1.4884, -0.6821, -1.3943],
          [-0.3098,  1.057

In [11]:
output = unfold(x)
print(output.shape) 
# 6개 요소 x 5 채널 = 30 요소. 그리고 kernel이 (3,4) feature map 에 매칭되는 게 4번.
print(output)

torch.Size([2, 30, 4])
tensor([[[-1.5223, -0.2952,  0.0066, -0.6348],
         [-0.2952,  0.3551, -0.6348, -1.2648],
         [ 0.3551,  0.2865, -1.2648, -0.3411],
         [ 0.0066, -0.6348,  0.1455,  0.6420],
         [-0.6348, -1.2648,  0.6420, -1.2262],
         [-1.2648, -0.3411, -1.2262,  0.6646],
         [ 0.5227,  0.0675,  1.7125,  0.7122],
         [ 0.0675,  2.2939,  0.7122, -0.6256],
         [ 2.2939, -0.9430, -0.6256,  1.7609],
         [ 1.7125,  0.7122,  0.6527,  2.1585],
         [ 0.7122, -0.6256,  2.1585, -1.9699],
         [-0.6256,  1.7609, -1.9699, -1.4661],
         [-0.4349,  0.4933, -0.6547,  0.1629],
         [ 0.4933, -1.7456,  0.1629, -0.0294],
         [-1.7456,  0.0696, -0.0294,  1.2622],
         [-0.6547,  0.1629, -0.2090, -0.0254],
         [ 0.1629, -0.0294, -0.0254, -0.5135],
         [-0.0294,  1.2622, -0.5135, -1.0149],
         [-1.9260,  1.1769,  0.6423, -0.6941],
         [ 1.1769, -1.1372, -0.6941, -1.6763],
         [-1.1372, -0.2015, -1.6763, 

2. convolution is equivalent with unfold + matrix multiplication + fold

In [12]:
inp = torch.randn(1, 3, 10, 12)

In [13]:
w = torch.randn(2, 3, 4, 5)

In [15]:
inp_unf = torch.nn.functional.unfold(inp, (4, 5))
inp_unf.shape # 20 x 3 = 60

torch.Size([1, 60, 56])

In [17]:
inp_unf.transpose(1, 2).shape

torch.Size([1, 56, 60])

In [22]:
w.size(0)

2

In [23]:
w.view(w.size(0), -1).shape

torch.Size([2, 60])

In [24]:
w.view(w.size(0), -1).t().shape

torch.Size([60, 2])

In [25]:
inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).shape

torch.Size([1, 56, 2])

In [29]:
out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
out_unf.shape

torch.Size([1, 2, 56])

In [30]:
help(torch.nn.functional.fold)

Help on function fold in module torch.nn.functional:

fold(input, output_size, kernel_size, dilation=1, padding=0, stride=1)
    Combines an array of sliding local blocks into a large containing
    tensor.
    
    .. warning::
        Currently, only 4-D output tensors (batched image-like tensors) are
        supported.
    
    See :class:`torch.nn.Fold` for details



In [31]:
out = torch.nn.functional.fold(out_unf, (7, 8), (1, 1))
out.shape

torch.Size([1, 2, 7, 8])

In [33]:
out2 = torch.nn.functional.conv2d(inp, w)
out2.shape

torch.Size([1, 2, 7, 8])